In [111]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn 
import streamlit as st
from ydata_profiling import ProfileReport
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import roc_curve, auc
import xgboost
import pickle
import joblib

In [112]:
data = pd.read_csv('Expresso_churn_dataset.csv')
data

,user_id,REGION,TENURE,MONTANT,FREQUENCE_RECH,REVENUE,ARPU_SEGMENT,FREQUENCE,DATA_VOLUME,ON_NET,ORANGE,TIGO,ZONE1,ZONE2,MRG,REGULARITY,TOP_PACK,FREQ_TOP_PACK,CHURN
0,00000bfd7d50f01092811bc0c8d7b0d6fe7c3596,FATICK,K > 24 month,4250.0,15.0,4251.0,1417.0,17.0,4.0,388.0,46.0,1.0,1.0,2.0,NO,54,On net 200F=Unlimited _call24H,8.0,0
1,00000cb4a5d760de88fecb38e2f71b7bec52e834,NaN,I 18-21 month,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,4,NaN,NaN,1
2,00001654a9d9f96303d9969d0a4a851714a4bb57,NaN,K > 24 month,3600.0,2.0,1020.0,340.0,2.0,NaN,90.0,46.0,7.0,NaN,NaN,NO,17,On-net 1000F=10MilF;10d,1.0,0
3,00001dd6fa45f7ba044bd5d84937be464ce78ac2,DAKAR,K > 24 month,13500.0,15.0,13502.0,4501.0,18.0,43804.0,41.0,102.0,2.0,NaN,NaN,NO,62,"Data:1000F=5GB,7d",11.0,0
4,000028d9e13a595abe061f9b58f3d76ab907850f,DAKAR,K > 24 month,1000.0,1.0,985.0,328.0,1.0,NaN,39.0,24.0,NaN,NaN,NaN,NO,11,Mixt 250F=Unlimited_call24H,2.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,7cb5a227517f04b2d30f40e80b8207957fba7761,SAINT-LOUIS,K > 24 month,9000.0,17.0,9000.0,3000.0,16.0,0.0,344.0,181.0,NaN,1.0,NaN,NO,56,MIXT:500F= 2500F on net _2500F off net;2d,9.0,0
1048571,7cb5a30c0494d3f3c17675ceff78970d4e83748a,NaN,K > 24 month,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,2,NaN,NaN,1
1048572,7cb5aca1f1be878196785b9041b2d0acd85c3280,KAFFRINE,K > 24 month,10000.0,10.0,10004.0,3335.0,16.0,25542.0,22.0,58.0,1.0,NaN,NaN,NO,62,"Data:1000F=2GB,30d",10.0,0
1048573,7cb5c3d45abd64d3b13ded0bd60804e130a54cb6,DIOURBEL,K > 24 month,1000.0,1.0,984.0,328.0,2.0,5.0,6.0,1.0,NaN,NaN,NaN,NO,8,IVR Echat_Daily_50F,2.0,0


In [113]:
data.drop(["FREQUENCE_RECH","ARPU_SEGMENT","FREQUENCE","MONTANT","DATA_VOLUME","ON_NET","ORANGE","TIGO","ZONE1","ZONE2","TOP_PACK","FREQ_TOP_PACK","user_id"],axis =1, inplace=True)

In [114]:
#Display general information about the dataset
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 6 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   REGION      635141 non-null   object 
 1   TENURE      1048575 non-null  object 
 2   REVENUE     694797 non-null   float64
 3   MRG         1048575 non-null  object 
 4   REGULARITY  1048575 non-null  int64  
 5   CHURN       1048575 non-null  int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 48.0+ MB


In [115]:
data.describe()

,REVENUE,REGULARITY,CHURN
count,694797.000000,1.048575e+06,1.048575e+06
mean,5514.798879,2.803957e+01,1.876823e-01
std,7199.856252,2.229285e+01,3.904585e-01
min,1.000000,1.000000e+00,0.000000e+00
25%,1000.000000,6.000000e+00,0.000000e+00
50%,3000.000000,2.400000e+01,0.000000e+00
75%,7387.000000,5.100000e+01,0.000000e+00
max,532177.000000,6.200000e+01,1.000000e+00


In [116]:
#Create a pandas profiling reports to gain insights into the dataset
from ydata_profiling import ProfileReport
profile = ProfileReport(data, title = 'Expresso Report')
profile.to_file('expresso.html')
profile.to_notebook_iframe()


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [117]:
data.isnull().sum()

REGION        413434
TENURE             0
REVENUE       353778
MRG                0
REGULARITY         0
CHURN              0
dtype: int64

In [118]:
#Handle Missing and corrupted values
data['REGION'].fillna(data['REGION'].mode()[0],inplace=True)
#data['MONTANT'].fillna(data['MONTANT'].mean(),inplace=True)
data['REVENUE'].fillna(data['REVENUE'].mean(),inplace=True)

In [119]:
#Remove duplicates, if they exist
data.duplicated().sum()

640823

In [120]:
data.drop_duplicates(inplace=True)

In [121]:
data.duplicated().sum()

0

In [122]:
data['CHURN'].unique()

array([0, 1], dtype=int64)

In [123]:
data['TENURE'].unique()

array(['K > 24 month', 'I 18-21 month', 'G 12-15 month', 'H 15-18 month',
       'J 21-24 month', 'F 9-12 month', 'D 3-6 month', 'E 6-9 month'],
      dtype=object)

In [124]:
tenure_encoder = {'K > 24 month': 24,
                  'I 18-21 month': 18,
                  'G 12-15 month':12, 
                  'H 15-18 month': 15,
                  'J 21-24 month':21,
                  'F 9-12 month':9, 
                  'D 3-6 month':3, 
                  'E 6-9 month':6 }

In [125]:
data['TENURE'] = data['TENURE'].map(tenure_encoder)

In [126]:
#Encode categorical features
cat = data.select_dtypes('object').columns
cat

Index(['REGION', 'MRG'], dtype='object')

In [127]:
le = LabelEncoder()
le.fit(data['REGION'])
le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
le_name_mapping

{'DAKAR': 0,
 'DIOURBEL': 1,
 'FATICK': 2,
 'KAFFRINE': 3,
 'KAOLACK': 4,
 'KEDOUGOU': 5,
 'KOLDA': 6,
 'LOUGA': 7,
 'MATAM': 8,
 'SAINT-LOUIS': 9,
 'SEDHIOU': 10,
 'TAMBACOUNDA': 11,
 'THIES': 12,
 'ZIGUINCHOR': 13}

In [128]:
le = LabelEncoder()
data['REGION'] = le.fit_transform(data['REGION'])

In [129]:
mrge = LabelEncoder()
data['MRG']= mrge.fit_transform(data['MRG'])

In [130]:
#saving the Label Encoder into a file using joblib
file = 'joblib_Encoder.sav'
joblib.dump(le,file)

['joblib_Encoder.sav']

In [131]:
y = data['CHURN']
x = data.drop('CHURN', axis =1)

In [132]:
#Based on the previous data exploration train and test a machine learning classifier
x_train,x_test,y_train,y_test = train_test_split(x,y, test_size=0.2,random_state=42)



In [133]:
#using Decision Tree Classifier
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier()
train = tree.fit(x_train,y_train)
y_pred =tree.predict(x_test)
print('Accuracy:',accuracy_score(y_test, y_pred))



Accuracy: 0.9085480251621685


In [134]:
#using Random Forest
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=10)
clf.fit(x_train,y_train)
clf_pred = clf.predict(x_test)
print('Accuracy =', accuracy_score(y_test,clf_pred))

Accuracy = 0.9162609900553028


In [135]:
#using xgboost
from xgboost import XGBClassifier
bst = XGBClassifier(n_estimators=2, max_depth=2, learning_rate=1, objective='binary:logistic')
bst.fit(x_train, y_train)
preds = bst.predict(x_test)
print('Accuracy =', accuracy_score(y_test,preds))

Accuracy = 0.9481674044462974


In [136]:
from sklearn.metrics import classification_report, confusion_matrix
cn = confusion_matrix(y_test, preds)
cn

array([[77324,     0],
       [ 4227,     0]], dtype=int64)

In [137]:
import joblib 

# save model with joblib 
filename = 'joblib_clf.sav'
joblib.dump(clf,filename)

['joblib_clf.sav']